# Model Correction

```{margin}
```{important}
This tutorial assumes that `BuildingMOTIF` has already been installed in the local environment.
```

The purpose of this tutorial is to learn how to fix a model that fails validation using BuildingMOTIF Templates to automate the correction process.

## Setup

Like the previous tutorial, we'll create an in-memory BuildingMOTIF instance, load the model, and load some libraries. We'll also load the manifest from the previous tutorial.

In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BRICK # import this to make writing URIs easier

# in-memory instance
bm = BuildingMOTIF("sqlite://")

# create the namespace for the building
BLDG = Namespace('urn:bldg/')

# create the building model
model = Model.create(BLDG, description="This is a test model for a simple building")

# load tutorial 1 model
model.graph.parse("tutorial2_model.ttl", format="ttl")

# load in some libraries
brick = Library.load(ontology_graph="../../libraries/brick/Brick-subset.ttl")
constraints = Library.load(ontology_graph="../../buildingmotif/resources/constraints.ttl")
manifest = Library.load(ontology_graph="tutorial2_manifest.ttl")
g36 = Library.load(directory="../../libraries/ashrae/guideline36")

ModuleNotFoundError: No module named 'rdflib'

## Model Validation

Let's validate the model again to see what's causing the failure.

In [ ]:
# gather shape collections into a list for ease of use
shape_collections = [
    brick.get_shape_collection(),
    constraints.get_shape_collection(),
    manifest.get_shape_collection(),
    g36.get_shape_collection(),
]

# pass a list of shape collections to .validate()
validation_result = model.validate(shape_collections)
print(f"Model is valid? {validation_result.valid}")

# print reasons
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

## Model Correction with Templates

The model is failing because the AHU doesn't have the minimum number of supply fans associated with it. We *could* add the fan explicitly by adding those triples to the model like we've done previously, but we can also ask BuildingMOTIF to generate new templates that explicitly prompt us for the missing information. We can also take a closer look at the first autogenerated template

In [ ]:
# create a new library to hold these generated templates
generated_templates = Library.create("my-autogenerated-templates")
for diff in validation_result.diffset:
    diff.resolve(generated_templates)

# print some of the autogenerated template
for templ in generated_templates.get_templates():
    templ = templ.inline_dependencies()
    print(f"Name (autogenerated): {templ.name}")
    print(f"Parameters (autogenerated): {templ.parameters}")
    print("Template body (autogenerated):")
    print(templ.body.serialize())
    print('-' * 79)
    break # just to demonstrate the first piece of output

In this case, the generated templates are fairly simple. They require an input for the name of the supply fan and the names of several missing points. We can loop through each of these generated templates and create the names. Here, we are creating arbitrary names for the points but in a real setting you would likely pull the equipment or point names from an external source like a Building Information Model or BACnet network[^1] (see future tutorials for how to do this!) Another challenge is the fact that we already have a supply fan in the model. Here, we can take advantage of the fact that the name of the fan in the existing model are just the name of the AHU wtih the `-Fan` postfix. The name of the AHU is in the generated templates (see above) so we can just pull out the name of the AHU, add the postfix, and use that as the value for the `name` parameter.

[^1]: https://bacnet.org/

In [ ]:
from buildingmotif.namespaces import BRICK
for templ in generated_templates.get_templates():
    templ = templ.inline_dependencies()
    # get name of AHU from template body
    ahu_name = next(templ.body.subjects(predicate=BRICK.hasPart))
    # generate the name of the supply fan
    supply_fan_name = ahu_name + '-Fan'

    # we know from the exploration above that each template has
    # 1 parameter called 'name', which we assign to the name of the supply fan
    bindings = {
        "name": supply_fan_name,
    }
    # the rest of the parameters have random names, so let's generate
    # random URIs for those by putting the name of the parameter at the end
    # of the fan's name
    for p in templ.parameters - {'name'}:
        print(p)
        bindings[p] = supply_fan_name + p
    supply_fan = templ.evaluate(bindings)
    model.add_graph(supply_fan)
    print(f"Added supply fan {supply_fan_name}")

We use the same code as before to ask BuildingMOTIF if the model is now valid:

In [ ]:
validation_result = model.validate(shape_collections)
print(f"Model is valid? {validation_result.valid}")

Success! The model is valid with respect to the targeted use case, i.e. the model can support the high-performance sequences of operation for single zone VAV AHUs from ASHRAE Guideline 36. Let's take a look at the validated model and save it for use in future tutorials.

In [ ]:
# print model
print(model.graph.serialize())

#save model
model.graph.serialize(destination="tutorial3_model.ttl")